# CPR appropriation with policy gradient

This notebook contains actual Harvest trainings for each implemented policy gradient method. The environment in use is a custom implementation of Harvest.

## Pre-requisites

The cells down below install and import the necessary libraries to successfully run the notebook examples.

In [1]:
import sys
sys.path.append('../')

In [3]:

!pip install -r ../init/requirements.txt
!pip install -e ../src/gym_cpr_grid

Obtaining file:///home/somz/cpr-appropriation/src/gym_cpr_grid
  Preparing metadata (setup.py) ... done


  Attempting uninstall: gym-cpr-grid
    Found existing installation: gym-cpr-grid 0.0.1
    Uninstalling gym-cpr-grid-0.0.1:
      Successfully uninstalled gym-cpr-grid-0.0.1
  Running setup.py develop for gym-cpr-grid


In [12]:
import time

import numpy as np
import gym
import torch
import matplotlib.pyplot as plt
from IPython import display

from src import memory, models, policies



## Utilities

The cell down below defines the environment, along with common variables to be used throughout the notebook.

In [13]:
env = gym.make(
    'gym_cpr_grid:CPRGridEnv-v0', 
    n_agents=1, 
    grid_width=25, 
    grid_height=7,
    tagging_ability=False,
    gifting_mechanism=None,
    initial_resource_probability=0.2,
    fov_squares_front=21,
    global_obs=False
)

In [14]:
observation_space_size = env.observation_space_size()
action_space_size = env.action_space_size()
epochs = 1000
steps_per_epoch = 1000
minibatch_size = 100
save_every = 200
hidden_sizes = [32, 32]
#checkpoints_path = "../checkpoints"
render_every = None
std_returns = True
std_advs = False
clip_gradient_norm = None
policy_lr = 3e-4
baseline_lr = 1e-3
wandb_config = {
    "api_key": "18638505c190d67f1040f6945fe54ab128a99dde",
    "project": "cpr-appropriation",
    "entity": "somz",
}

## VPG

This section deals with training a set of Harvest agents using our custom Vanilla Policy Gradient implementation.

In [16]:
vpg_policy_nn = models.MLP(observation_space_size, hidden_sizes, action_space_size)
vpg_baseline_nn = models.MLP(observation_space_size, hidden_sizes, 1, log_softmax=False)
vpg_policy = policies.VPGPolicy(env, vpg_policy_nn, baseline_nn=vpg_baseline_nn)
vpg_policy.train(
    epochs,
    steps_per_epoch,
    minibatch_size,
    enable_wandb=True,
    wandb_config={**wandb_config, "group": "VPG"},
    save_every=save_every,
    checkpoints_path=checkpoints_path,
    render_every=render_every,
    std_returns=std_returns,
    std_advs=std_advs,
    clip_gradient_norm=clip_gradient_norm,
    policy_lr=policy_lr,
    baseline_lr=baseline_lr
)

2021-10-19 14:51:04.944 | DEBUG    | src.models:__init__:58 - Model summary: MLP(
  (mlp): Sequential(
    (0): Linear(in_features=1323, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=9, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
)
2021-10-19 14:51:04.950 | DEBUG    | src.models:__init__:58 - Model summary: MLP(
  (mlp): Sequential(
    (0): Linear(in_features=1323, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
)


efficiency,20.0
equality,1.0
sustainability,267.3
peace,nan
loss,-36.87967
mean_return,20.0
_runtime,21
_timestamp,1634669464
_step,0


efficiency,▁
equality,▁
sustainability,▁
peace,
loss,▁
mean_return,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2021-10-19 14:51:12.164 | INFO     | src.policies:train:126 - Epoch 1 / 1000
2021-10-19 14:51:12.165 | INFO     | src.policies:collect_trajectories:230 - Episode 1
2021-10-19 14:51:13.754 | DEBUG    | src.policies:execute_episode:436 - Early stopping, all agents done
2021-10-19 14:51:13.756 | INFO     | src.policies:collect_trajectories:241 - Episode infos: {'efficiency': 20.0, 'equality': 1.0, 'sustainability': 267.3, 'peace': nan}
2021-10-19 14:51:13.757 | INFO     | src.policies:collect_trajectories:246 - Mean episode return: 20.0
2021-10-19 14:51:13.758 | INFO     | src.policies:collect_trajectories:247 - Last 100 episodes mean return: 20.0
2021-10-19 14:51:14.531 | INFO     | src.policies:train:167 - Mini-batch 1 / 10
2021-10-19 14:51:14.534 | INFO     | src.policies:minibatch_update:291 - Losses: {'policy_loss': -0.022714532911777496, 'baseline_loss': 0.9563030004501343, 'total_loss': 0.45543697476387024}
2021-10-19 14:51:14.536 | INFO     | src.policies:minibatch_update:297 - Po

AssertionError: The given path does not exist

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/somz/cpr-appropriation/2kps6xu9/file_stream
NoneType: None
wandb: ERROR Error while calling W&B API: Error 1062: Duplicate entry '503936-2kps6xu9' for key 'PRIMARY' (<Response [409]>)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/somz/cpr-appropriation/2kps6xu9/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/somz/cpr-appropriation/32mjg777/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/somz/cpr-appropriation/32mjg777/file_stream
NoneTy

## TRPO

This section deals with training a set of Harvest agents using our custom Trust Region Policy Optimization implementation.

In [ ]:
beta = 1.0
kl_target = 0.01

In [ ]:
trpo_policy_nn = models.MLP(observation_space_size, hidden_sizes, action_space_size)
trpo_baseline_nn = models.MLP(observation_space_size, hidden_sizes, 1, log_softmax=False)
trpo_policy = policies.TRPOPolicy(env, trpo_policy_nn, trpo_baseline_nn, beta=beta, kl_target=kl_target)
trpo_policy.train(
    epochs,
    steps_per_epoch,
    minibatch_size,
    enable_wandb=True,
    wandb_config={**wandb_config, "group": "TRPO"},
    save_every=save_every,
    checkpoints_path=checkpoints_path,
    render_every=render_every,
    std_returns=std_returns,
    std_advs=std_advs,
    clip_gradient_norm=clip_gradient_norm,
    policy_lr=policy_lr,
    baseline_lr=baseline_lr
)

## PPO

This section deals with training a set of Harvest agents using our custom Proximal Policy Optimization implementation.

In [ ]:
alpha = 0.5
beta = 0.01
eps = 0.2

In [ ]:
ppo_policy_nn = models.MLP(observation_space_size, hidden_sizes, action_space_size)
ppo_baseline_nn = models.MLP(observation_space_size, hidden_sizes, 1, log_softmax=False)
ppo_policy = policies.PPOPolicy(env, ppo_policy_nn, ppo_baseline_nn, alpha=alpha, beta=beta, eps=eps)
ppo_policy.train(
    epochs,
    steps_per_epoch,
    minibatch_size,
    enable_wandb=True,
    wandb_config={**wandb_config, "group": "PPO"},
    save_every=save_every,
    checkpoints_path=checkpoints_path,
    render_every=render_every,
    std_returns=std_returns,
    std_advs=std_advs,
    clip_gradient_norm=clip_gradient_norm,
    policy_lr=policy_lr,
    baseline_lr=baseline_lr
)

## Evaluation

In this section we are evaluating one of the trained models on the Harvest environment and visualizing results through custom rendering functions.

In [ ]:
policy = ppo_policy
checkpoint = ""
policy.load(checkpoint)

In [ ]:
policy.policy_nn.eval()
policy.baseline_nn.eval()
observations = env.reset()
prev_actions = {h: None for h in range(env.n_agents)}
for _ in range(env._max_episode_steps):
    env.render()
    action_dict = dict()
    for agent_handle in range(env.n_agents):
        legal_actions = env.get_legal_actions(agent_handle)
        probs = np.exp(
            ppo_policy.policy_nn(
                torch.tensor(observations[agent_handle], dtype=torch.float32), 
                torch.tensor(legal_actions, dtype=torch.int64)
            ).cpu().detach().numpy()
        )
        action = np.argmax(probs)
        if prev_actions[agent_handle] == 0 and action == 1:
            probs[1] = 0
        if prev_actions[agent_handle] == 1 and action == 0:
            probs[0] = 0
        if prev_actions[agent_handle] == 2 and action == 3:
            probs[3] = 0
        if prev_actions[agent_handle] == 3 and action == 2:
            probs[2] = 0
        if prev_actions[agent_handle] == 4 and action == 5:
            probs[5] = 0
        if prev_actions[agent_handle] == 5 and action == 4:
            probs[4] = 0
        if prev_actions[agent_handle] == action:
            probs[action] == 0
        action = np.argmax(probs)
        prev_actions[agent_handle] = action

    print(env.agent_positions)
    action_dict = {0: action}
    print(action_dict)
    observations, rewards, dones, infos = env.step(action_dict)
    print(infos)
    display.clear_output(wait=True)
    #time.sleep(3)
env.close()